<a href="https://colab.research.google.com/github/js08080/AIAccelarator/blob/main/RAG_Complete_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Implementation with LlamaIndex and LanceDB

This notebook demonstrates a complete RAG (Retrieval Augmented Generation) implementation using LlamaIndex and LanceDB. We'll explore three different approaches:

1. **Vector Search Only** - Fast retrieval without LLM generation
2. **HuggingFace API Integration** - Cloud-based LLM with authentication
3. **Local LLM with Ollama** - Complete local solution

## Overview

The notebook covers:
- Data loading and preparation from HuggingFace datasets
- Vector store setup with LanceDB
- Embedding generation with HuggingFace models
- Three different query approaches with increasing complexity
- Utility functions for table exploration and optimization

## 1. Install Required Dependencies

In [ ]:
# # Install all required packages
# !pip install llama-index llama-index-vector-stores-lancedb llama-index-embeddings-huggingface llama-index-llms-huggingface-api lancedb datasets -q

# # Additional packages for local LLM and utilities
# !pip install llama-index-llms-ollama requests -q

## 2. Import Libraries and Setup

In [ ]:
import os
import lancedb
import subprocess
import requests
import time
from pathlib import Path
from datasets import load_dataset

# LlamaIndex core components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# Embedding and vector store
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.lancedb import LanceDBVectorStore

# LLM integrations
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.ollama import Ollama

# Async support for notebooks
import nest_asyncio
nest_asyncio.apply()

print("All libraries imported successfully")

: 

In [ ]:
from llama_index.vector_stores.chroma

## 3. Data Preparation and Loading

In [ ]:
def prepare_data(num_samples=100):
    """
    Load dataset and create document files
    """
    print(f"Loading {num_samples} personas from dataset...")

    # Load the personas dataset
    dataset = load_dataset("dvilasuero/finepersonas-v0.1-tiny", split="train")

    # Create data directory
    Path("data").mkdir(parents=True, exist_ok=True)

    # Save personas as text files and create Document objects
    documents = []
    for i, persona in enumerate(dataset.select(range(min(num_samples, len(dataset))))):
        # Create Document objects for LlamaIndex
        doc = Document(
            text=persona["persona"],
            metadata={
                "persona_id": i,
                "source": "finepersonas-dataset"
            }
        )
        documents.append(doc)

        # Optionally save to files as well
        with open(Path("data") / f"persona_{i}.txt", "w", encoding="utf-8") as f:
            f.write(persona["persona"])

    print(f"Prepared {len(documents)} documents")
    return documents

# Load the data
documents = prepare_data(num_samples=100)

Loading 100 personas from dataset...
Prepared 100 documents


## 4. LanceDB Vector Store Setup

In [ ]:
def setup_lancedb_store(table_name="personas_rag"):
    """
    Initialize LanceDB and create/connect to a table
    """
    print("Setting up LanceDB connection...")

    # Create or connect to LanceDB
    db = lancedb.connect("./lancedb_data")

    # LlamaIndex will handle table creation with proper schema
    print(f"Connected to LanceDB, table: {table_name}")

    return db, table_name

# Setup database connection
db, table_name = setup_lancedb_store()

Setting up LanceDB connection...
Connected to LanceDB, table: personas_rag


## 5. Vector Embeddings and Ingestion Pipeline

In [ ]:
async def create_and_populate_index(documents, db, table_name):
    """
    Create ingestion pipeline and populate LanceDB with embeddings
    """
    print("Creating embedding model and ingestion pipeline...")

    # Initialize embedding model
    embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5"
    )

    # Create LanceDB vector store
    vector_store = LanceDBVectorStore(
        uri="./lancedb_data",
        table_name=table_name,
        mode="overwrite"  # overwrite existing table
    )

    # Create ingestion pipeline
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(chunk_size=512, chunk_overlap=20),
            embed_model,
        ],
        vector_store=vector_store,
    )

    print("Processing documents and creating embeddings...")
    # Run the pipeline to process documents and store in LanceDB
    nodes = await pipeline.arun(documents=documents)
    print(f"Successfully processed {len(nodes)} text chunks")

    return vector_store, embed_model

# Create embeddings and populate vector store
vector_store, embed_model = await create_and_populate_index(documents, db, table_name)

Creating embedding model and ingestion pipeline...
Processing documents and creating embeddings...
Successfully processed 100 text chunks


In [ ]:
vector_store.to_dict()

{'stores_text': True,
 'is_embedding_query': True,
 'flat_metadata': True,
 'uri': './lancedb_data',
 'vector_column_name': 'vector',
 'nprobes': 20,
 'refine_factor': None,
 'text_key': 'text',
 'doc_id_key': 'doc_id',
 'api_key': None,
 'region': None,
 'mode': 'overwrite',
 'query_type': 'vector',
 'overfetch_factor': 1,
 'class_name': 'base_component'}

## 6. Option 1: Vector Search Only (No LLM)

This approach provides fast document retrieval without LLM generation. Perfect for finding relevant content quickly.

In [ ]:
def perform_vector_search(db, table_name, query_text, embed_model, top_k=5):
    """
    Perform direct vector search on LanceDB
    """
    # Get query embedding
    query_embedding = embed_model.get_text_embedding(query_text)

    # Open table and perform search
    table = db.open_table(table_name)
    results = table.search(query_embedding).limit(top_k).to_pandas()

    return results

def test_vector_search():
    """
    Test vector search functionality with sample queries
    """
    print("Testing Vector Search (No LLM needed)")
    print("=" * 50)

    # Test queries
    queries = [
        "technology and artificial intelligence expert",
        "teacher educator professor",
        "environment climate sustainability",
        "art culture heritage creative"
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 30)

        # Perform search
        results = perform_vector_search(db, table_name, query, embed_model, top_k=3)

        for idx, row in results.iterrows():
            score = row.get('_distance', 'N/A')
            text = row.get('text', 'N/A')

            # Format score
            if isinstance(score, (int, float)):
                score_str = f"{score:.3f}"
            else:
                score_str = str(score)

            print(f"\nResult {idx + 1} (Score: {score_str}):")
            print(f"{text[:200]}...")

# Run vector search test
test_vector_search()

Testing Vector Search (No LLM needed)

Query: technology and artificial intelligence expert
------------------------------

Result 1 (Score: 0.589):
A computer scientist or electronics engineer researching alternative sustainable materials for neuromorphic computing and memristor development or A science journalist covering emerging technologies a...

Result 2 (Score: 0.589):
A computer scientist or electronics engineer researching alternative sustainable materials for neuromorphic computing and memristor development or A science journalist covering emerging technologies a...

Result 3 (Score: 0.589):
A computer scientist or electronics engineer researching alternative sustainable materials for neuromorphic computing and memristor development or A science journalist covering emerging technologies a...

Query: teacher educator professor
------------------------------

Result 1 (Score: 0.577):
An English language arts teacher with a focus on upper elementary education....

Result 2 (Scor

## 7. Option 2: RAG with HuggingFace API

This approach uses HuggingFace's cloud API for LLM generation. Requires API token authentication.

In [ ]:
# Set your HuggingFace API token here
# Get your free token from: https://huggingface.co/settings/tokens
os.environ["HUGGINGFACE_API_KEY"] = ""  # Replace with your actual token

def create_query_engine(vector_store, embed_model, llm=None):
    """
    Create a query engine from the vector store
    """
    # Create index from vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model
    )

    # Setup LLM if provided
    query_engine_kwargs = {}
    if llm:
        query_engine_kwargs['llm'] = llm

    # Create query engine
    query_engine = index.as_query_engine(
        response_mode="tree_summarize",
        **query_engine_kwargs
    )

    return query_engine

def query_rag(query_engine, question):
    """
    Query the RAG system and return response
    """
    response = query_engine.query(question)
    return response

async def test_huggingface_rag():
    """
    Test RAG with HuggingFace API
    """
    print("Testing RAG with HuggingFace API")
    print("=" * 40)

    try:
        # Initialize HuggingFace LLM with authentication
        llm = HuggingFaceInferenceAPI(
            model_name=  "HuggingFaceH4/zephyr-7b-beta", #"HuggingFaceTB/SmolLM3-3B",
            token=os.environ.get("HUGGINGFACE_API_KEY")
        )

        # Create query engine
        query_engine = create_query_engine(vector_store, embed_model, llm)

        # Test queries
        queries = [
            "Find personas interested in technology and AI",
            "Who are the educators or teachers in the dataset?",
            "Describe personas working with environmental topics"
        ]

        for query in queries:
            print(f"\nQuery: {query}")
            print("-" * 30)

            try:
                response = query_rag(query_engine, query)
                print(f"Response: {response}")
            except Exception as e:
                print(f"Error: {e}")

    except Exception as e:
        print(f"Setup error: {e}")
        print("Make sure to set your HuggingFace API token above")

# Uncomment the line below after setting your API token
await test_huggingface_rag()

Testing RAG with HuggingFace API

Query: Find personas interested in technology and AI
------------------------------
Response: <think>
Okay, let's see. The user is asking for personas interested in technology and AI. The context provided is about an aerospace engineer or astrobiologist from the finepersonas-dataset. So, I need to figure out if this persona is interested in technology and AI.

First, aerospace engineers work on designing and developing aircraft, spacecraft, and missiles. They use a lot of advanced technologies, including AI for simulations, data analysis, and autonomous systems. Astrobiologists study the origin, evolution, distribution, and future of life in the universe. They might use AI for analyzing data from space missions, like searching for signs of life on other planets.

So, both roles involve working with technology and AI. The aerospace engineer would definitely be interested in AI for their projects, and the astrobiologist would use AI in their research. Th

## 8. Option 3: RAG with Local LLM (Ollama)

This approach uses a completely local LLM setup. No internet required after initial setup.

In [ ]:
def check_ollama_installed():
    """Check if Ollama is installed"""
    try:
        result = subprocess.run(["ollama", "--version"],
                              capture_output=True, text=True, shell=True)
        if result.returncode == 0:
            print(f"Ollama is installed: {result.stdout.strip()}")
            return True
    except FileNotFoundError:
        pass

    print("Ollama is not installed or not in PATH")
    return False

def download_ollama():
    """Download Ollama installer for Windows"""
    print("Downloading Ollama for Windows...")

    url = "https://ollama.com/download/OllamaSetup.exe"
    response = requests.get(url)

    installer_path = Path("OllamaSetup.exe")
    with open(installer_path, "wb") as f:
        f.write(response.content)

    print("Ollama downloaded successfully!")
    print("Please run the installer manually and then continue.")
    print(f"Installer location: {installer_path.absolute()}")

    return installer_path

def start_ollama_service():
    """Start Ollama service"""
    try:
        print("Starting Ollama service...")
        subprocess.Popen(["ollama", "serve"], shell=True)
        time.sleep(3)
        print("Ollama service started!")
        return True
    except Exception as e:
        print(f"Failed to start Ollama: {e}")
        return False

def pull_ollama_model(model_name="llama3.2:1b"):
    """Pull a lightweight model for local inference"""
    try:
        print(f"Pulling model: {model_name}")
        result = subprocess.run(["ollama", "pull", model_name],
                              capture_output=True, text=True, shell=True)
        if result.returncode == 0:
            print(f"Model {model_name} pulled successfully!")
            return True
        else:
            print(f"Failed to pull model: {result.stderr}")
            return False
    except Exception as e:
        print(f"Error pulling model: {e}")
        return False

def setup_ollama():
    """Complete Ollama setup"""
    if not check_ollama_installed():
        print("Ollama needs to be installed.")
        download_ollama()
        return False

    if not start_ollama_service():
        return False

    if not pull_ollama_model("llama3.2:1b"):
        return False

    print("Ollama setup complete!")
    return True

# Check Ollama installation
check_ollama_installed()

In [ ]:
async def test_local_llm_rag():
    """
    Test RAG with local Ollama LLM
    """
    print("Testing RAG with Local LLM (Ollama)")
    print("=" * 40)

    try:
        # Initialize local Ollama LLM
        llm = Ollama(
            model="llama3.2:1b",
            base_url="http://localhost:11434",
            request_timeout=60.0
        )

        # Create query engine
        query_engine = create_query_engine(vector_store, embed_model, llm)

        # Test queries
        queries = [
            "Find personas interested in technology and AI",
            "Who are the educators or teachers in the dataset?",
            "Describe personas working with environmental topics"
        ]

        for query in queries:
            print(f"\nQuery: {query}")
            print("-" * 30)

            try:
                response = query_rag(query_engine, query)
                print(f"Response: {response}")
            except Exception as e:
                print(f"Error: {e}")
                print("Make sure Ollama is running with: ollama serve")

    except Exception as e:
        print(f"Setup error: {e}")
        print("Make sure Ollama is installed and running")

# Uncomment after Ollama setup is complete
# await test_local_llm_rag()

## 9. Utility Functions and Advanced Features

In [ ]:
def explore_lancedb_table(db, table_name):
    """
    Explore the structure and content of the LanceDB table
    """
    try:
        table = db.open_table(table_name)

        print("Table Schema:")
        print(table.schema)

        print(f"\nTotal records: {table.count_rows()}")

        print("\nSample records:")
        df = table.to_pandas().head()
        print(df)

        return table
    except Exception as e:
        print(f"Error exploring table: {e}")
        return None

def create_filtered_query_engine(db, table_name, embed_model, filter_dict=None):
    """
    Create a query engine with metadata filtering capabilities
    """
    from llama_index.core.vector_stores import MetadataFilters, MetadataFilter, FilterOperator

    # Reconnect to existing table
    vector_store = LanceDBVectorStore(
        uri="./lancedb_data",
        table_name=table_name,
        mode="read"
    )

    # Create index
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model
    )

    # Create query engine with filters if provided
    if filter_dict:
        filters = MetadataFilters(
            filters=[
                MetadataFilter(
                    key=key,
                    value=value,
                    operator=FilterOperator.EQ
                ) for key, value in filter_dict.items()
            ]
        )
        query_engine = index.as_query_engine(
            filters=filters,
            response_mode="tree_summarize"
        )
    else:
        query_engine = index.as_query_engine(
            response_mode="tree_summarize"
        )

    return query_engine

async def batch_process_documents(documents, batch_size=50):
    """
    Process documents in batches for large datasets
    """
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        table_name = f"personas_batch_{i//batch_size}"

        vector_store = LanceDBVectorStore(
            uri="./lancedb_data",
            table_name=table_name,
            mode="overwrite"
        )

        pipeline = IngestionPipeline(
            transformations=[
                SentenceSplitter(chunk_size=512, chunk_overlap=20),
                embed_model,
            ],
            vector_store=vector_store,
        )

        nodes = await pipeline.arun(documents=batch)
        print(f"Processed batch {i//batch_size + 1}: {len(nodes)} nodes")

def show_usage_examples():
    """
    Display usage examples for different scenarios
    """
    print("Usage Examples:")
    print("=" * 30)

    print("\n1. Vector Search Only:")
    print("   test_vector_search()")

    print("\n2. HuggingFace API RAG:")
    print("   # Set API token first")
    print("   os.environ['HUGGINGFACE_API_KEY'] = 'your_token'")
    print("   await test_huggingface_rag()")

    print("\n3. Local LLM RAG:")
    print("   # Install and setup Ollama first")
    print("   setup_ollama()")
    print("   await test_local_llm_rag()")

    print("\n4. Explore Database:")
    print("   explore_lancedb_table(db, table_name)")

# Show usage examples
show_usage_examples()

Usage Examples:

1. Vector Search Only:
   test_vector_search()

2. HuggingFace API RAG:
   # Set API token first
   os.environ['HUGGINGFACE_API_KEY'] = 'your_token'
   await test_huggingface_rag()

3. Local LLM RAG:
   # Install and setup Ollama first
   setup_ollama()
   await test_local_llm_rag()

4. Explore Database:
   explore_lancedb_table(db, table_name)


In [ ]:
explore_lancedb_table(db, table_name)

Table Schema:
id: string
doc_id: string
vector: fixed_size_list<item: float>[384]
  child 0, item: float
text: string
metadata: struct<_node_content: string, _node_type: string, doc_id: string, document_id: string, persona_id: i (... 41 chars omitted)
  child 0, _node_content: string
  child 1, _node_type: string
  child 2, doc_id: string
  child 3, document_id: string
  child 4, persona_id: int64
  child 5, ref_doc_id: string
  child 6, source: string

Total records: 100

Sample records:
                                     id                                doc_id  \
0  60cb883b-dc1b-473e-bf1b-911526643e61  4ba97bc4-f109-4c80-8fe9-748c1194e69e   
1  06694472-6c44-4656-9fde-675afd87fe26  105dbc32-cb12-4b26-a63a-dbfdedf37d81   
2  10150826-af1c-4f3d-94d7-e1d16752bf89  873190dd-82aa-4f8a-9585-f8a4d2e35973   
3  292bcb9f-7796-4fd3-b6eb-3d6edd36639f  2a590ad4-bcab-4a76-9dde-114d46c98160   
4  67c222f2-3931-4951-8395-64f8b7dd39fc  152cb28f-bdee-4318-acde-9233165b32c2   

                   

LanceTable(name='personas_rag', version=1, _conn=LanceDBConnection(uri='/Users/divijbajaj/Library/CloudStorage/OneDrive-Personal/Personal_Projects/agents/lancedb_data'))

## Summary

This notebook provides three complete RAG implementation approaches:

### Option 1: Vector Search Only
- **Best for**: Fast document retrieval, no generation needed
- **Advantages**: Very fast, no API costs, no setup complexity
- **Use case**: Finding relevant documents, initial exploration

### Option 2: HuggingFace API
- **Best for**: High-quality responses with cloud LLMs
- **Advantages**: Latest models, no local resources needed
- **Requirements**: HuggingFace API token
- **Use case**: Production applications with budget for API calls

### Option 3: Local LLM (Ollama)
- **Best for**: Complete privacy, no internet dependency
- **Advantages**: No API costs, full control, offline capability
- **Requirements**: Ollama installation, local compute resources
- **Use case**: Private data, cost-sensitive applications

Choose the approach that best fits your needs!